# Mittelfluss aus Investitionstätigkeit

## Einlesen der erforderlichen Paket und Daten

Auch hier muss als Erstes die pandas Library importiert werden. Anschliessend
müssen die Daten einem pandas DataFrame zugewiesen werden.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv', index_col=0)

## Rekonstruktion des Mittelflusses aus Investitionstätigkeit

Für die Rekonstruktion des Mittelflusses aus Investitionstätigkeit kann auf die
'Formel' zurückgegriffen werden, welche im 
[Schweizerischen Kontenrahmen KMU](https://www.kmu.admin.ch/dam/kmu/de/dokumente/savoir-pratique/Finances/240812%20Schulkontenrahmen%20VEB%20-%20DE.pdf.download.pdf/240812%20Schulkontenrahmen%20VEB%20-%20DE.pdf)
abgedruckt ist. Die 'Formel' ist unten aufgeführt.

$–$ Investitionen in Finanzanlagen  
$+$ Devestitionen von Finanzanlagen  
$–$ Investitionen in Beteiligungen  
$+$ Devestitionen von Beteiligungen  
$–$ Investitionen in mobile Sachanlagen  
$+$ Devestitionen von mobilen Sachanlagen  
$–$ Investitionen in immobile Sachanlagen  
$+$ Devestitionen von immobilen Sachanlagen  
$–$ Investitionen in immaterielle Werte  
$+$ Devestitionen von immateriellen Werten  

Die meisten Schulaufgaben gehen allerdings von bloss zwei Positionen im Anlagevermögen aus:
Mobilien und Immobilien. Aus diesem Grund wird im vorliegenden Beispiel
nur mit den Positionen Mobilien und Immobilien gearbeitet. Die zu Grunde
liegenden Überlegungen sind jedoch auch auf die anderen Positionen anwendbar.

### Rekonstruktion der Abschreibungen auf Mobilien

Um die erforderlichen Berechnungen zu planen, lohnt es sich unter Umständen, den
genauen Ablauf der Berechnungen in einem Flussdiagramm darzustellen.

```{mermaid}
graph TD
    %% Main Function
    A[Start: investitionsbereich] --> B[Initialize Dictionary 'investitionen']
    B --> C[Calculate Mobile Investments]
    B --> D[Calculate Immobile Investments]
    
    %% Mobile Investment Branch
    C --> C1[Call _get_invest_mob]
    C1 --> C2{Check wb_mob exists?}
    C2 -->|Yes| C3[Calculate with WB]
    C2 -->|No| C4[Calculate without WB]
    
    %% Mobile Investment Calculation with WB
    C3 --> C5[Get Mobile Depreciation]
    C5 --> C6[Calculate Delta WB Mobile]
    C6 --> C7[Sum Mobile Investment]
    
    %% Mobile Investment Calculation without WB
    C4 --> C8[Get Total Depreciation]
    C8 --> C7
    
    %% Immobile Investment Branch
    D --> D1[Call _get_invest_immo]
    D1 --> D2{Check wb_immo exists?}
    D2 -->|Yes| D3[Calculate with WB]
    D2 -->|No| D4[Calculate without WB]
    
    %% Immobile Investment Calculation
    D3 --> D5[Get Immobile Depreciation]
    D5 --> D6[Calculate Delta WB Immobile]
    D6 --> D7[Sum Immobile Investment]
    D4 --> D5
    
    %% Final Calculations and Output
    C7 --> E[Sum Total Investments]
    D7 --> E
    E --> F[Format Output]
    F --> G[Print Results]
    G --> H[End]
    
    %% Helper Functions
    subgraph Helper Functions
        H1[_get_total_absch]
        H2[_get_delta_wb_mob]
        H3[_get_delta_wb_immo]
        H4[_check_wb_mob]
        H5[_check_wb_immo]
        H6[_get_absch_mob]
        H7[_get_absch_immo]
    end
    
    %% Relationships between Helper Functions
    C8 --> H1
    C6 --> H2
    D6 --> H3
    C2 --> H4
    D2 --> H5
    C5 --> H6
    D5 --> H7
    
    %% Style Definitions
    classDef process fill:#f9f,stroke:#333,stroke-width:2px;
    classDef decision fill:#ffd,stroke:#333,stroke-width:2px;
    classDef helper fill:#dff,stroke:#333,stroke-width:2px;
    
    %% Apply Styles
    class A,B,C,D,E,F,G process;
    class C2,D2 decision;
    class H1,H2,H3,H4,H5,H6,H7 helper;
```


### Definition der erforderlichen Helferfunktionen

Helferfunktionen werden in Python mit einem einleitenden Unterstrich definiert.

#### `_check_wb_mob`

In [3]:
def _check_wb_mob(ledger: pd.DataFrame) -> bool:
    wb_mob = ledger.index.intersection([1509, 1519, 1529, 1539, 1549])
    
    return len(wb_mob) > 0

#### `_check_wb_immo`

In [4]:
def _check_wb_immo(ledger: pd.DataFrame) -> bool:
        
    return 1609 in ledger.index

#### `_get_total_absch`

In [5]:
def _get_total_absch(ledger: pd.DataFrame) -> float:
    """
    Get the total amount of Abschreibungen from the ledger.
    """
    konti = [i for i in range(6800, 6900)]
    total_absch = 0
    for konto in konti:
        if konto in ledger.index:
            total_absch += ledger.loc[konto, 'Abschluss'] * -1
    
    return total_absch

#### `_get_delta_wb_mob`

In [6]:
def _get_delta_wb_mob(ledger: pd.DataFrame) -> float:
    """
    Get the delta of the wb_mob from the ledger.
    """
    wb_mob = ledger.index.intersection([1509, 1519, 1529, 1539, 1549])
    
    if len(wb_mob) == 0:
        return 0
    
    delta_wb_mob = 0
    for konto in wb_mob:
        delta_wb_mob += ledger.loc[konto, 'Abschluss'] - ledger.loc[konto, 'Eroeffnung']
    
    return delta_wb_mob

#### `_get_delta_wb_immo`

In [7]:
def _get_delta_wb_immo(ledger: pd.DataFrame) -> float:
    delta_wb_immo = ledger.loc[1609, 'Abschluss'] - ledger.loc[1609, 'Eroeffnung']
    return delta_wb_immo

#### `_get_absch_mob`

In [8]:
def _get_absch_mob(ledger: pd.DataFrame) -> float:
    absch = _get_total_absch(ledger)    
 
    if _check_wb_mob(ledger):
        absch_mob = _get_delta_wb_mob(ledger)
    else:
        absch_mob =  absch - _get_delta_wb_immo(ledger)
    return absch_mob

#### `_get_absch_mob_wb`

In [9]:
def _get_absch_mob_wb(ledger: pd.DataFrame) -> float:
    konti = ledger.index.intersection([1509, 1519, 1529, 1539, 1549])
    absch_mob = 0
    for konto in konti:
        absch_mob += ledger.loc[konto, 'Abschluss'] - ledger.loc[konto, 'Eroeffnung']
    return absch_mob * -1  # Negativ, weil es eine Erhöhung des WB ist

#### `_get_absch_immo`

In [10]:
def _get_absch_immo(ledger: pd.DataFrame) -> float:
    absch = ledger.loc[6800, 'Abschluss'] * -1
    absch_mob = _get_absch_mob(ledger) * -1    
    absch_immo = absch - absch_mob    
    return absch_immo

#### `_get_invest_mob`

In [11]:
def _get_invest_mob(ledger: pd.DataFrame) -> float:
    mob = ledger.index.intersection([1500, 1510, 1520, 1530, 1540])
    invest_mob = 0
    for m in mob:
        invest_mob += ledger.loc[m, 'Abschluss'] - ledger.loc[m, 'Eroeffnung']
        
    if not _check_wb_mob(ledger):
        invest_mob += _get_absch_mob(ledger)
        
    return invest_mob

#### `_get_infest_immo`

In [12]:
def _get_invest_immo(ledger: pd.DataFrame) -> float:
    delta_immo = ledger.loc[1600, 'Abschluss'] - ledger.loc[1600, 'Eroeffnung']
    
    if not _check_wb_immo(ledger):
        delta_immo += _get_absch_immo(ledger)
    
    return delta_immo

### Funktion `investitionsbereich()`

Die Funktion `investitionsbereich()` ist die Zusammenfassung aller vorher
definierten Hilfsfunktionen. Mit Ihr wird der Mittelfluss aus
Investitionstätigkeit aus dem aufbereiteten DataFrame extrahiert.

In [13]:
def investitionsbereich(ledger: pd.DataFrame):
    investitionen = {}
    investitionen['Mobilien'] = _get_invest_mob(ledger)
    investitionen['Immobilien'] = _get_invest_immo(ledger)
    
    breite = 50
    titel = 'Mittelfluss aus Investitionen\n\n'
    
    for index, value in investitionen.items():
        mitte = breite - len(index) -len(str(value))
        breite_links = len(index) + mitte // 2
        breite_rechts = len(str(value)) + mitte // 2
        if breite_links + breite_rechts < breite:
            breite_links += 1
        titel += f"{index:<{breite_links}}{value:>{breite_rechts}}\n"
        
    mitte = breite - len('Nettoinvestitionen') - len(str(investitionen['Mobilien'] + investitionen['Immobilien']))
    breite_links = len('Nettoinvestitionen') + mitte // 2
    breite_rechts = len(str(investitionen['Mobilien'] + investitionen['Immobilien'])) + mitte // 2
    if breite_links + breite_rechts < breite:
        breite_links += 1
    inv = 'Nettoinvestitionen'
    inv_zahl = str(investitionen['Mobilien'] + investitionen['Immobilien'])
    titel += f"{inv:<{breite_links}}{inv_zahl:>{breite_rechts}}\n"
        
    print(titel)
    

In [14]:
investitionsbereich(df)

Mittelfluss aus Investitionen

Mobilien                                      20.0
Immobilien                                     0.0
Nettoinvestitionen                            20.0

